In [2]:
import os
import requests

output_dir        = "output_chunks"
audio_file_path   = 'downloaded_audio.mp3'
noise_reduced_dir = "noise_reduced_chunks"
audio_url         = "https://storage.googleapis.com/radiofilez/english/saturday/kfm_93_3-2022-11-26_T11.00.01.mp3"

if not os.path.exists(audio_file_path):
    response = requests.get(audio_url, allow_redirects=True)
    if response.status_code == 200:
        with open(audio_file_path, 'wb') as f:
            f.write(response.content)
        print(f"Audio file downloaded successfully: {audio_file_path}")
    else:
        print(f"Failed to download audio file: {response.status_code}")

# demucs vs spleeter

In [ ]:
!pip install demucs spleeter diffq

In [2]:
import os
os.system(f'demucs --two-stems vocals -n htdemucs --mp3 {audio_file_path}')

35072

In [ ]:
import subprocess
response = subprocess.run(
    args = [
        'demucs',
        '--two-stems', 'vocals',
        '-n', 'htdemucs',
        # '-n', 'mdx_extra_q',
        # '-n', 'mdx_q',
        '--mp3',
        f'{audio_file_path}'
    ],
    text = True,
    capture_output = True
)
print(response.stdout)
print(response.stderr)

Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs
Separating track downloaded_audio.mp3


  0%|                                                                     | 0.0/3586.0499999999997 [00:00<?, ?seconds/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,

  0%|                                                            | 5.85/3586.0499999999997 [00:04<45:56,  1.30seconds/s]
  0%|▏                                                           | 11.7/3586.0499999999997 [00:04<20:19,  2.93seconds/s]
  0%|▏                                             | 17.549999999999997/3586.0499999999997 [

In [4]:
import subprocess
response = subprocess.run(
    args = ['spleeter',
        'separate',
        '-p', 'spleeter:2stems',
        '-o', 'spleeter_output',
        f'{audio_file_path}'
    ],
    text = True,
    capture_output = True
)
print(response.stdout)

INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
INFO:spleeter:File spleeter_output/downloaded_audio/vocals.wav written succesfully
INFO:spleeter:File spleeter_output/downloaded_audio/accompaniment.wav written succesfully



# webrtcvad vs PyAudioAnalysis vs inaSpeechSegmenter vs vad.Energyvad

---



In [5]:
! pip install pyAudioAnalysis webrtcvad eyed3 pydub inaSpeechSegmenter vad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 14.2 MB/s eta 0:00:00
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264373 sha256=cfbd76666a824e415014bdcb51cf959a8f5882255ce92483cbadb982c1795872
  Stored in di

## webrtcvad

In [34]:
import webrtcvad
import contextlib
import wave
from torchaudio import load

vocals_path = 'spleeter_output/downloaded_audio/vocals.wav'

# with wave.open(vocals_path,mode='rb') as wf:

with contextlib.closing(wave.open(vocals_path, 'rb')) as wf:
    num_channels = wf.getnchannels()
    # assert num_channels == 1
    sample_width = wf.getsampwidth()
    # assert sample_width == 2
    sample_rate = wf.getframerate()
    # assert sample_rate in (8000, 16000, 32000, 48000)
    pcm_data = wf.readframes(wf.getnframes())
pcm_data, sample_rate

vad = webrtcvad.Vad()

vad.is_speech(pcm_data,sample_rate)
# wave_read_obj = wave.open(vocals_path,mode='rb')
# params = wave_read_obj.getparams()
# frames = wave_read_obj.readframes(params.nframes)
# params.nframes

Error: Error while processing frame

## pyaudioanalysis

In [ ]:
from pyAudioAnalysis import audioTrainTest as aT
# from pyAudioAnalysis import audioSegmentation as aS
# aT.extract_features_and_train(["classifierData/music","classifierData/speech"], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "svmSMtemp", False)
aT.file_classification("/content/downloaded_audio.mp3", "svmSMtemp","svm")

fileClassification: input model_name not found!


(-1, -1, -1)

## inaspeech segmenter

In [3]:
from inaSpeechSegmenter import Segmenter

seg = Segmenter(
    detect_gender = False,
    vad_engine = 'sm', # speech music noise smn
    batch_size = 512,
)
segmentation = seg(audio_file_path)
segmentation

349/349 - 337s - 337s/epoch - 966ms/step


[('speech', 0.0, 15.620000000000001),
 ('noEnergy', 15.620000000000001, 16.080000000000002),
 ('speech', 16.080000000000002, 50.76),
 ('music', 50.76, 86.22),
 ('speech', 86.22, 90.94),
 ('music', 90.94, 97.46000000000001),
 ('speech', 97.46000000000001, 101.78),
 ('music', 101.78, 108.12),
 ('speech', 108.12, 110.08),
 ('music', 110.08, 127.32000000000001),
 ('speech', 127.32000000000001, 256.72),
 ('music', 256.72, 263.84000000000003),
 ('noEnergy', 263.84000000000003, 264.86),
 ('speech', 264.86, 318.78000000000003),
 ('music', 318.78000000000003, 323.92),
 ('speech', 323.92, 342.7),
 ('music', 342.7, 359.3),
 ('speech', 359.3, 399.72),
 ('music', 399.72, 401.96000000000004),
 ('speech', 401.96000000000004, 440.38),
 ('noEnergy', 440.38, 441.64),
 ('speech', 441.64, 473.28000000000003),
 ('noEnergy', 473.28000000000003, 474.14),
 ('music', 474.14, 476.42),
 ('speech', 476.42, 498.72),
 ('noEnergy', 498.72, 500.72),
 ('speech', 500.72, 534.5),
 ('noEnergy', 534.5, 535.0600000000001),

## vad.energyVad

In [4]:
from vad.energy_vad import EnergyVAD
from torchaudio import load, save

waveform, sample_rate = load(audio_file_path)

vad = EnergyVAD()
vad_output = vad(waveform)

# save("example_vad.wav", new_waveform, sample_rate)
vad_output

array([0., 0., 0., ..., 1., 1., 1.])

# pipeline

In [5]:
os.system('pip install openai-whisper')

0

In [6]:
import whisper
model = whisper.load_model(
    name = "base",
    # device = device,
)

os.makedirs(output_dir, exist_ok=True)
os.makedirs(noise_reduced_dir, exist_ok=True)

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 88.1MiB/s]


In [7]:
import io
from scipy.io.wavfile import read
def audiosegment_to_librosawav(segment):
    out_f = io.BytesIO()
    segment.export(out_f, format='wav')
    out_f.seek(0)
    rate, data = read(out_f)
    return rate, data

In [8]:
from pydub import AudioSegment
def transcribe_audio_segment(segment_file_path, model=model):
    result = model.transcribe(
        audio =segment_file_path,
        word_timestamps = True
    )
    return result["text"]

In [9]:
os.system('pip install noisereduce')

0

In [10]:
import numpy as np
import noisereduce as nr
def split_audio_file(audio_file_path, output_dir, segment_duration_s=150):
    audio = AudioSegment.from_mp3(audio_file_path)
    num_segments = int(audio.duration_seconds / segment_duration_s) + 1

    for i in range(num_segments):
        start_time = i * segment_duration_s * 1000
        end_time = min((i + 1) * segment_duration_s * 1000, audio.duration_seconds * 1000)

        segment = audio[start_time:end_time]
        segment_file_path = os.path.join(output_dir, f"segment_{i + 1}.mp3")
        segment.export(segment_file_path, format="mp3")

        rate, data = audiosegment_to_librosawav(segment)
        if segment.channels == 2:
            data = np.mean(data, axis=1)
        reduced_noise = nr.reduce_noise(y=data, sr=rate,prop_decrease=0.8)

        if reduced_noise.dtype != np.int16:
            reduced_noise = np.int16(reduced_noise / np.max(np.abs(reduced_noise)) * 32767)

        noise_reduced_segment = AudioSegment(reduced_noise.tobytes(), frame_rate=rate, sample_width=reduced_noise.dtype.itemsize, channels=1)
        noise_reduced_segment_path = os.path.join(noise_reduced_dir, f"segment_{i + 1}_reduced_noise.mp3")
        noise_reduced_segment.export(noise_reduced_segment_path, format="mp3")
        yield segment_file_path,noise_reduced_segment_path

In [11]:
for segment_file_path, noise_reduced_segment_path in split_audio_file(audio_file_path, output_dir):
    print()
    transcription = transcribe_audio_segment(noise_reduced_segment_path)
    print(transcription)


 2022. Did you know that teenage pregnancies contribute 20% of infadds and 28% of maternal deaths? Join the conversation to speak out and help in ending child marriages and teenage pregnancies in Uganda. This message is brought to you by Action 8 in partnership with UNIFPA UNICEF, Ministry of Gender, Labor and Social Development. Push down the finest cuisine with the purest water that has a test cracked by nature. Brinzori Minowater is your trusted source to help you stay healthy, hydrated and refreshed. Brinzori is from a mirror which underground sauce that delivers nature's goodness in a bottle. Nature creates it. Brinzori bottles its goodness. Save a life naturally with Uganda's purest natural Minowater. Easy Sunday. I found a love. A whole new, a whole new, infatastic boy. Easy Sunday. Only your favorite music. Online 3-3 UKFM. Everything is double, double, double, double, double, double, double, double, double, double, and even Christmas double happiness is back! What a gosh! Lis

# Whisper x

In [ ]:
! pip install pytorch==2.0.0 torchaudio==2.0.0 pytorch-cuda=11.8 -c
! pip install protobuf==3.18.*
! pytorch -c nvidia
! pip install git+https://github.com/m-bain/whisperx.git --upgrade

In [18]:
# import whisperx
import gc
from pprint import pprint

In [9]:
device       = "cuda"
audio_file   = 'downloaded_audio.mp3'
batch_size   = 32
compute_type = "float16"
model        = whisperx.load_model("base", device, compute_type=compute_type)
audio        = whisperx.load_audio(audio_file)

result       = model.transcribe(audio, batch_size=batch_size)
pprint(result["segments"])

# model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
# result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

# pprint(result["segments"])

No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0+cu121. Bad things might happen unless you revert torch to 1.x.
Detected language: en (0.92) in first 30s of audio...
[{'end': 24.428,
  'start': 0.009,
  'text': ' 2022 Did you know that teenage pregnancies contribute 20% of '
          'infant deaths and 28% of maternal deaths? Join the conversation to '
          'speak out and help in ending child marriages and teenage '
          'pregnancies in Uganda. This message is brought to you by Action 8 '
          'in partnership with UNIFPA UNICEF, Ministry of Gender, Labor and '
          'Social Development.'},
 {'end': 54.94,
  'start': 25.964,
  'text': ' Push down the finest cuisine with the purest water that has a '
          'taste cracked by nature. Branzori Minowater is your trusted source '
          'to help you stay healthy, hydrated and refresh

In [ ]:
pprint(result)

In [1]:
! gsutil ls "gs://radiofilez/english"

gs://radiofilez/english/
gs://radiofilez/english/friday/
gs://radiofilez/english/monday/
gs://radiofilez/english/saturday/
gs://radiofilez/english/sunday/
gs://radiofilez/english/thursday/
gs://radiofilez/english/tuesday/
gs://radiofilez/english/wednesday/


In [6]:
import subprocess
import pandas as pd

df = pd.DataFrame(columns=['day','link','filename','radio','id','date','hour'])

days = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

for day in days:
    response = subprocess.run(
        args = ['gsutil', 'ls', f"gs://radiofilez/english/{day}"],
        text = True,
        capture_output=True
    )
    links = response.stdout.split('\n')
    for link in links:
        meta = link.split('_')
        if len(meta)<=1: continue
        filename = link.split('/')[-1]
        radio=meta[0].split('/')[-1]
        id   =meta[-3]
        date =meta[-2][2:]
        hour =meta[-1].split('.')[0][1:]
        df = pd.concat(
            [df,pd.DataFrame(data={
                'day'  :day,
                'link' : link,
                'filename':filename,
                'radio':radio,
                'id'   :id,
                'date' :date,
                'hour' :hour
            },index=[0])],
            ignore_index=True
        )
df

,day,link,filename,radio,id,date,hour
0,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T10.00.01.mp3,cp,91,2022-11-21,10
1,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T11.00.01.mp3,cp,91,2022-11-21,11
2,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T12.00.01.mp3,cp,91,2022-11-21,12
3,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T13.00.01.mp3,cp,91,2022-11-21,13
4,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T14.00.01.mp3,cp,91,2022-11-21,14
...,...,...,...,...,...,...,...
11129,sunday,gs://radiofilez/english/sunday/sanyu_88_2-2023...,sanyu_88_2-2023-05-21_T19.00.01.mp3,sanyu,88,2023-05-21,19
11130,sunday,gs://radiofilez/english/sunday/sanyu_88_2-2023...,sanyu_88_2-2023-05-21_T20.00.01.mp3,sanyu,88,2023-05-21,20
11131,sunday,gs://radiofilez/english/sunday/sanyu_88_2-2023...,sanyu_88_2-2023-05-21_T21.00.01.mp3,sanyu,88,2023-05-21,21
11132,sunday,gs://radiofilez/english/sunday/sanyu_88_2-2023...,sanyu_88_2-2023-05-21_T22.00.01.mp3,sanyu,88,2023-05-21,22


In [9]:
ten_links = df.head(10)
ten_links

,day,link,filename,radio,id,date,hour
0,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T10.00.01.mp3,cp,91,2022-11-21,10
1,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T11.00.01.mp3,cp,91,2022-11-21,11
2,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T12.00.01.mp3,cp,91,2022-11-21,12
3,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T13.00.01.mp3,cp,91,2022-11-21,13
4,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T14.00.01.mp3,cp,91,2022-11-21,14
5,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T15.00.01.mp3,cp,91,2022-11-21,15
6,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T16.00.01.mp3,cp,91,2022-11-21,16
7,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T17.00.01.mp3,cp,91,2022-11-21,17
8,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T18.00.01.mp3,cp,91,2022-11-21,18
9,monday,gs://radiofilez/english/monday/cp_91_3-2022-11...,cp_91_3-2022-11-21_T19.00.01.mp3,cp,91,2022-11-21,19


In [11]:
import os
for index in range(10):
    link = df.link[index]
    name = df.filename[index]
    os.system(f'gsutil -m cp -r "{link}" .')

    log_transcription(filename = name)


    os.system(f'rm -rf {name}')
